In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
symbol = "chow"
test = 1

if test:
    start = "2010-01-01"
    end = "2018-12-30"
else:
    start = "2019-01-01"
    end = "2019-05-22"
    
#symbol_file = open("mai.txt","r")
symbol_file = open("set_my_universe.txt","r")
symbols = [x.strip('\n') for x in symbol_file.readlines()]
data_path="../torodata/data/"
pdf_path="./pdf/"
volume_multiply = 2
commission = 0.003
for symbol in symbols:
    if symbol == "COM7":
        symbol = "_com7"
    df = pd.read_csv(data_path+symbol+".csv", index_col="Date", parse_dates=True)
    df.drop(columns=['Low','Adj Close'], inplace=True)
    df = df[start:end]

    setindex = pd.read_csv(data_path+"set.csv", index_col="Date", parse_dates=True)
    setindex = setindex[start:end]
    setindex['Setindex_Ret'] = setindex['Close'].pct_change()*100
    setindex.drop(columns=['Open','High','Low','Close','Adj Close', 'Volume'], inplace=True)
    df = df.merge(setindex, how='left',left_on = df.index, right_on = setindex.index, left_index=True)
    df.drop(columns=['key_0'], inplace=True)
    
    df['ClosetoCloseRet'] = (df['Close'].pct_change()*100) - commission
    df['NetSettleRet'] = ((df['Close']-df['Open'])/df['Open']*100) - commission
    df['ClosetoOpenRet'] = ((df['Open'].shift(-1) - df['Close'])/df['Close']*100) - commission
    df['OpentoOpenRet'] = ((df['Open'].shift(-1) - df['Open'])/df['Open']*100) - commission
    df['Vol_Sig'] = np.where(df.Volume > df.Volume.shift()*volume_multiply , 1, 0)
    df['Return_ClosetoCloseRet'] = df.Vol_Sig * df.ClosetoCloseRet.shift(-1)
    df['Return_ClosetoOpenRet'] = df.Vol_Sig * df.ClosetoOpenRet.shift(-1)
    df['Return_NetSettleRet'] = df.Vol_Sig * df.NetSettleRet.shift(-1)
    df['Return_OpentoOpenRet'] = df.Vol_Sig * df.OpentoOpenRet.shift(-1)
    setindex_ret = df['Setindex_Ret'].cumsum()[-3]
    stock_ret = df['Return_ClosetoOpenRet'].cumsum()[-3]
    if setindex_ret > stock_ret:
        continue
    if stock_ret < 5:
        continue
    print(symbol)
    df['Return_ClosetoCloseRet'].cumsum().plot(figsize=(15,8),legend=True, title=symbol.upper())
    df['Return_ClosetoOpenRet'].cumsum().plot(figsize=(15,8),legend=True)
    df['Return_NetSettleRet'].cumsum().plot(figsize=(15,8),legend=True)
    df['Return_OpentoOpenRet'].cumsum().plot(figsize=(15,8),legend=True)
    df['Setindex_Ret'].cumsum().plot(figsize=(15,8),legend=True)
    #df[df['Vol_Sig']>0]['Vol_Sig'].plot(figsize=(15,8),style="x")
    plt.savefig(pdf_path+symbol+'.pdf')
    plt.clf()
#print("Completed!")    

ACC
AJA
AKR
AMANAH
ANAN
AQUA
ASIAN
ASIMAR
B
BIG
CCET
CI
CKP
COTTO
CPT
CWT
DCON
DEMCO
EASON
EE
EKH
EVER
FN
GEL
GGC
GJS
GREEN
ICHI
IFS
INET
INGRS
JMART
JTS
LOXLEY
NSI
NVD
PAF
PERM
PK
PRAKIT
PRM
PSH
PT
PYLON
RS
SAM
SAMCO
SCI
SDC
SEAFCO
SKE
SLP
SMT
SQ
SUSCO
SYNTEC
TCC
TCMC
TEAM
TGPRO
TH
THE
TKN
TOG
TPOLY
TRITN
TSE
TSR
U
UV
VNT
VPO
WAVE
WIN
WP


<Figure size 1080x576 with 0 Axes>

In [2]:
import os
from PyPDF2 import PdfFileMerger

x = [a for a in os.listdir("./pdf/") if a.endswith(".pdf")]

merger = PdfFileMerger()

for pdf in x:
    merger.append(open("./pdf/"+pdf, 'rb'))

with open("result.pdf", "wb") as fout:
    merger.write(fout)
fout.close()